# Jupyter Dashのデモ

In [ ]:
# japanize_matplotlibが未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install japanize-matplotlib

In [ ]:
# JupyterDash他が未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install jupyter_dash
#!pip install --upgrade plotly

In [ ]:
# Googleドライブがマウントされていない場合に、2行のコメント解除して本セルを単独で実行する
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# matplotlibで日本語を使えるようにする
import japanize_matplotlib

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
import dash_html_components as html
import dash_core_components as dcc
from dash import dash_table as dt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from dash_table import DataTable, FormatTemplate
from dash_table.Format import Format

## データファイルを読み込む

In [ ]:
 df1 = pd.read_excel('/content/drive/My Drive/Colab Notebooks/都道府県各種指標.xlsx',sheet_name='data')
 df1.head(3)

,年度,地域コード,都道府県,総人口,総人口（男）,総人口（女）,日本人人口,日本人人口（男）,日本人人口（女）,15歳未満人口,15歳未満人口（男）,15歳未満人口（女）,15～64歳人口,15～64歳人口（男）,15～64歳人口（女）,65歳以上人口,65歳以上人口（男）,65歳以上人口（女）,出生数,出生数（男）,出生数（女）,合計特殊出生率,死亡数,死亡数（男）,死亡数（女）,転入者数（日本人移動者）,転入者数（日本人移動者）（男）,転入者数（日本人移動者）（女）,転出者数（日本人移動者）,転出者数（日本人移動者）（男）,転出者数（日本人移動者）（女）,婚姻件数,離婚件数,年平均気温,最高気温（日最高気温の月平均の最高値）,最低気温（日最低気温の月平均の最低値）,降水日数（年間）,降水量（年間）,着工建築物数,着工建築物床面積,...,大学卒業者数,大学卒業者のうち進学者数,専修学校数,各種学校数,専修学校生徒数,各種学校生徒数,新規求職申込件数（一般）,月間有効求職者数（一般）,月間有効求人数（一般）,充足数（一般）,就職件数（一般）,県内就職件数（一般）,一般旅券発行件数,着工新設住宅戸数,着工新設持家数,着工新設貸家数,着工新設分譲住宅数,着工新設住宅床面積,着工新設持家床面積,着工新設分譲住宅床面積,着工新設貸家床面積,一般病院数,一般診療所数,歯科診療所数,保育所等数,保育所等定員数,保育所等利用待機児童数,保育所等在所児数,保育所等保育士数,消費支出（二人以上の世帯）,食料費（二人以上の世帯）,住居費（二人以上の世帯）,光熱・水道費（二人以上の世帯）,家具・家事用品費（二人以上の世帯）,被服及び履物費（二人以上の世帯）,保健医療費（二人以上の世帯）,交通・通信費（二人以上の世帯）,教育費（二人以上の世帯）,教養娯楽費（二人以上の世帯）,その他の消費支出（二人以上の世帯）
0,2018,R01000,北海道,5286000,2489000,2797000,5253000,2475000,2778000,577000,295000,282000,3052000,1502000,1551000,1656000,692000,964000,32642,16681,15961,1.27,64187,32757,31430,47269,28263,19006,55222,31407,23815,22916,9971,9.5,25.7,-7.6,157,1282.0,21437,5192636,...,16831,2112,168,49,25820,3780,200363,734530,870195,43961,44510,42819,111793,35761,11740,19535,4067,2934844,1445452,415385,1041957,486,3392,2905,1003,80564,129,76931,10221,281054,69044,19089,27012,8845,11517,11437,39308,10825,26991,56984
1,2017,R01000,北海道,5320000,2506000,2814000,5292000,2494000,2797000,588000,301000,288000,3099000,1523000,1576000,1632000,683000,950000,34040,17503,16537,1.29,62417,31995,30422,48561,29134,19427,55130,31448,23682,23960,10147,9.1,27.7,-7.0,146,1158.0,21430,5480381,...,16707,2176,169,52,26928,4119,211045,772601,856866,47555,48178,46307,107166,37062,11466,21052,4134,3014254,1417542,438020,1132127,493,3384,2934,979,77751,65,73231,10393,286698,69640,24217,25633,10550,11222,12242,39559,9837,26797,57003
2,2016,R01000,北海道,5352000,2521000,2830000,5327000,2511000,2816000,600000,306000,294000,3150000,1545000,1605000,1602000,670000,932000,35125,17888,17237,1.29,61906,32072,29834,48544,29032,19512,55418,31974,23444,24636,10476,9.3,28.3,-5.9,152,1360.0,21294,5286695,...,17198,2173,171,54,28512,4181,228449,827073,839594,51494,52194,50088,98811,37515,11312,21810,4018,3080563,1421197,430776,1201056,493,3380,2968,908,73405,94,70119,10202,287325,69445,17071,24438,9533,11106,12431,43681,10751,29305,59564


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564 entries, 0 to 563
Columns: 108 entries, 年度 to その他の消費支出（二人以上の世帯）
dtypes: float64(5), int64(101), object(2)
memory usage: 476.0+ KB


In [ ]:
# 外部スタイルシートを読み込む
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# JupyterDashの「箱」を作る
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
# フォーマットのテンプレートを利用する
#percentage = FormatTemplate.percentage(1)
# 表示する列を定義する
columns1 = [
    dict(id='年度',name='年度'),
    dict(id='都道府県',name='都道府県'),
    dict(id='総人口', name='総人口'),
    dict(id='年平均気温', name='年平均気温'),
    dict(id='標準価格（平均価格）（住宅地）', name='標準価格（平均価格）（住宅地）'),
    dict(id='県内就職件数（一般）', name='県内就職件数（一般）')
]

In [ ]:
# ドロップダウンリストを定義する
content1 = dcc.Dropdown(
    id = 'dropdown-for-year', # id名はコールバックで必要
    options = [{'label':i,'value':i} for i in df1['都道府県'].unique()], # 値を選択する列を指定
    value = '東京都' # デフォルト値を設定
)

In [ ]:
# データテーブルを定義する
content2 = html.Div(id='datatable-paging',children=[]) # id名はコールバックで必要

In [ ]:
# 画面のレイアウトを定義する
app.layout = html.Div([
                       content1,
                       content2
])

In [ ]:
# df101 = df1[df1['都道府県'] == '東京都]

In [ ]:
# データテーブルを描画する関数を定義する
# 一行おきに色のコントラストをつけたデザインにする
def create_dash_table(df):
  return dt.DataTable(
      data=df.to_dict('records'),
      columns=columns1,
      style_cell={'fontsize':20, 'font-family':'IPAexGothic'},
      style_cell_conditional=[
                              {
                                  'if': {'column_id':c},
                                  'textAlign': 'left'
                              } for c in ['年度','都道府県']
      ],
      style_data={
          'color':'black',
          'backgroundColor':'white'
      },
      style_data_conditional=[
                          {
                              'if': {'row_index':'odd'},
                              'backgroundColor': 'rgb(220,220,220)',
                          }
      ],
      style_header={
      'backgroundColor':'rgb(210,210,210)',
      'color':'black',
      'fontWeight':'bold'
      })

In [ ]:
# コールバックを定義する（ドロップダウンリストで選択された値をもとに表を作成する）
# どこから入力し、どこへ出力するかを指定する
@app.callback(
    Output(component_id='datatable-paging',component_property='children'),
    [Input(component_id='dropdown-for-year',component_property='value')]
)
def update_table(input_value): # 行間をあけずに関数部分を書く（関数名は何でも良い）
  df101 = df1[df1['都道府県'] == input_value] # 値を選択する列を指定
  df101.sort_values('年度', ascending=True,inplace=True)
  return create_dash_table(df101)

In [ ]:
# サーバーを起動する
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>